In [1]:
societat = 'idiada-automotive-technology'

In [2]:
!pip install xlsxwriter

In [3]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb

wget -N https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/118.0.5993.70/linux64/chromedriver-linux64.zip -P /tmp/
unzip -o /tmp/chromedriver-linux64.zip -d /tmp/
chmod +x /tmp/chromedriver-linux64/chromedriver
mv /tmp/chromedriver-linux64/chromedriver /usr/local/bin/chromedriver
pip install selenium chromedriver_autoinstaller

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 257 kB in 2s (111 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
52 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependenc

In [4]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
import chromedriver_autoinstaller

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # this is must
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chromedriver_autoinstaller.install()

driver = webdriver.Chrome(options=chrome_options)

In [5]:
import pandas as pd
import numpy as np
import time

In [6]:
#societat = 'forestal-catalana'

url = "https://www.empresia.es/empresa/"+societat+"/"

driver.get(url)

while True:
  try:
      # Executa JavaScript per fer clic al botó
      driver.execute_script("document.querySelector('#NextEventos > a').click();")
      print("S'ha fet clic al botó 'Ver más...'")

      # Espera fins que la pàgina es carregui (ajusta a les teves necessitats)
      time.sleep(5)

  except:
      print("Ja no surt el botó 'Ver más...'")
      break

S'ha fet clic al botó 'Ver más...'
S'ha fet clic al botó 'Ver más...'
Ja no surt el botó 'Ver más...'


In [7]:
from bs4 import BeautifulSoup
import re

RM = []

# Suposem que tens el contingut HTML en una variable 'html'
html = driver.page_source

soup = BeautifulSoup(html, 'html.parser')

# Troba tots els elements <div> amb la classe 'event-content'
event_contents = soup.find_all('div', {'class': 'event-content'})

for event_content in event_contents:
    # Troba l'element <a> amb 'onclick' que contingui 'verPub'
    link_element = event_content.find('a', onclick=re.compile(r"verPub"))
    if link_element:
        # Extreu l'atribut 'onclick'
        onclick_attr = link_element.get('onclick')

        # Extreu l'URL utilitzant una expressió regular
        match = re.search(r"verPub\('(.+?)'", onclick_attr)
        if match:
            link = match.group(1)
        else:
            link = None
    else:
        link = None

    full = re.search(r"H\s+(\w+)\s+(\d+)", str(event_content))

    if 'Empresa' in event_content.text:
        actual = "Nom antic o altra societat"
    else:
        actual = "Nom actual"

    # Troba la data dins de l'element <time> anterior
    time_element = event_content.find_previous('time')
    datetime = time_element['datetime'] if time_element else None

    RM.append([
        link,
        datetime,
        " ".join(event_content.text.split()),
        (full[0] if full else None),
        actual
    ])

# Mostra els resultats
for item in RM:
    print(item)


['https://www.boe.es/borme/dias/2023/08/11/pdfs/BORME-A-2023-153-43.pdf', '2023-08-11', "Ceses/Dimisiones.Publicación en el BORME de los siguientes ceses-dimisiones: ConsejerosAgencia Per A La Competitivitat De L'empresa (acciEmpresa De Promocio I Localitzacio Industrial De CSecretarioCasals Noguer AngelaFuente: Boletín Oficial del Registro Mercantil Fecha inscripción: 04/08/2023. Núm. 365660. T 3379, F 205, S 8, H T 18969, I/A 87", 'H T 18969', 'Nom antic o altra societat']
['https://www.boe.es/borme/dias/2023/08/11/pdfs/BORME-A-2023-153-43.pdf', '2023-08-11', "Nombramientos.Publicación en el BORME de los siguientes nombramientos: ConsejerosAgencia Per A La Competitivitat De L'empresa (acciEmpresa De Promocio I Localitzacio Industrial De CSecretarioArgiles Malonda EvaVicesecretarioContreras Castro JavierFuente: Boletín Oficial del Registro Mercantil Fecha inscripción: 04/08/2023. Núm. 365660. T 3379, F 205, S 8, H T 18969, I/A 87", 'H T 18969', 'Nom antic o altra societat']
['https://

In [8]:
# Troba l'element <a> amb el títol que comença per "Ver documento"
element = soup.find('a', title=re.compile(r'^Ver documento'))

# Extreu l'atribut 'onclick'
onclick_attr = element.get('onclick')

# Extreu l'URL utilitzant una expressió regular
match = re.search(r"verPub\('(.+?)'", onclick_attr)
if match:
    url = match.group(1)
    print("URL del target:", url)
else:
    print("No s'ha trobat l'URL")

URL del target: https://www.boe.es/borme/dias/2023/08/11/pdfs/BORME-A-2023-153-43.pdf


In [9]:
import pandas as pd

pd.DataFrame(RM)

,0,1,2,3,4
0,https://www.boe.es/borme/dias/2023/08/11/pdfs/...,2023-08-11,Ceses/Dimisiones.Publicación en el BORME de lo...,H T 18969,Nom antic o altra societat
1,https://www.boe.es/borme/dias/2023/08/11/pdfs/...,2023-08-11,Nombramientos.Publicación en el BORME de los s...,H T 18969,Nom antic o altra societat
2,https://www.boe.es/borme/dias/2023/08/11/pdfs/...,2023-08-11,Otros conceptos:Con fecha 25/01/2023 se inscri...,H T 18969,Nom actual
3,https://www.boe.es/borme/dias/2023/08/11/pdfs/...,2023-08-11,Ceses/Dimisiones. Con fecha 4 de agosto de 202...,H T 18969,Nom actual
4,https://www.boe.es/borme/dias/2023/08/11/pdfs/...,2023-08-11,Cancelaciones de oficio de nombramientos. Con ...,H T 18969,Nom actual
...,...,...,...,...,...
83,http://www.boe.es/borme/dias/2010/09/16/pdfs/B...,2010-09-16,Depósito de cuentas anuales En agosto del 2010...,None,Nom actual
84,http://www.boe.es/borme/dias/2009/11/17/pdfs/B...,2009-11-17,Reelecciones. Con fecha 5 de noviembre de 2009...,H T 18969,Nom actual
85,http://www.boe.es/borme/dias/2009/10/05/pdfs/B...,2009-10-05,Depósito de cuentas anuales En agosto del 2009...,None,Nom actual
86,http://www.boe.es/borme/dias/2009/05/08/pdfs/B...,2009-05-08,Ceses/Dimisiones.Publicación en el BORME de lo...,H T 18969,Nom actual


In [10]:
'''
import pandas as pd
pd.DataFrame(RM).to_excel(societat+'-empresia.xlsx', index=False)
'''

"\nimport pandas as pd\npd.DataFrame(RM).to_excel(societat+'-empresia.xlsx', index=False)\n"

In [11]:
'''
from google.colab import files
files.download(societat+'-empresia.xlsx')
'''

"\nfrom google.colab import files\nfiles.download(societat+'-empresia.xlsx')\n"

In [12]:
# buscar el div amb l'identificador 'organigrama'
organigrama_div = soup.find('div', {'id': 'organigrama'})

# buscar tots els elements 'li' dins del div amb l'identificador 'organigrama'

try:
  li_elements = organigrama_div.find_all('li')

  carrecs = []

# obtenir només el primer text de cada element 'li'
  for li in li_elements:
      # obtenir tots els continguts de l'element 'li'
      contents = li.contents

      # obtenir el primer text
      nom = str(contents[1].text).strip()
      carrec = str(contents[3]).strip()

      # imprimir el primer text
      carrecs.append([nom, carrec])

except:
  carrecs = []



In [13]:
pd.DataFrame(carrecs)

,0,1
0,Grasas Alsina Carles,Presid.
1,Agencia Per A La Competitivitat De L'empresa (...,Consej.
2,APPLUS EUSKADI HOLDING SL,Consej.
3,Grasas Alsina Carles,Consej.
4,APPLUS ENERGY SL,Consej.
5,APLLUS EUSKADI HOLDING SL,Consej.
6,Agencia De Suport A L'empresa Catalana Accio,Consej.
7,Empresa De Promocio I Localitzacio Industrial ...,Consej.
8,APPLUS ITEUVE TECHNOLOGY SL,Consej.
9,APPLUS SERVICES SA,Consej.


In [14]:
import datetime

avui = datetime.datetime.now()
avui = str(avui).split(" ")[0].replace("-", "")
'''
pd.DataFrame(carrecs).to_excel(avui+" "+societat+'-carrecs-empresia.xlsx', index=False)
'''

'\npd.DataFrame(carrecs).to_excel(avui+" "+societat+\'-carrecs-empresia.xlsx\', index=False)\n'

In [15]:
'''
files.download(avui+" "+societat+'-carrecs-empresia.xlsx')
'''

'\nfiles.download(avui+" "+societat+\'-carrecs-empresia.xlsx\')\n'

In [16]:
carrecs_hist = soup.find('table', {'id': 'EntidadRelacion'})
carrecs_hist = pd.read_html(str(carrecs_hist))[0]
carrecs_hist

,Entidad,Relación,Entidad/Relación,Desde,Hasta
0,APPLUS EUSKADI HOLDING SL,Consej.,APPLUS EUSKADI HOLDING SL Consej.,04/08/2023,NaN
1,Argiles Malonda Eva,Secr.,Argiles Malonda Eva Secr.,04/08/2023,NaN
2,Agencia Per A La Competitivitat De L'empresa (...,Consej.,Agencia Per A La Competitivitat De L'empresa (...,04/08/2023,NaN
3,Contreras Castro Javier,Vicesecr.,Contreras Castro Javier Vicesecr.,04/08/2023,NaN
4,Casals Noguer Angela,Secr.,Casals Noguer Angela Secr.,NaN,04/08/2023
5,Farran Porte Josep Maria,Apod.,Farran Porte Josep Maria Apod.,13/07/2020,NaN
6,Grasas Alsina Carles,Presid.,Grasas Alsina Carles Presid.,13/07/2020,NaN
7,Grasas Alsina Carles,Consej.,Grasas Alsina Carles Consej.,13/07/2020,NaN
8,APPLUS SERVICES SA,Presid.,APPLUS SERVICES SA Presid.,28/04/2009,13/07/2020
9,Grasas Alsina Carles,Dir. General,Grasas Alsina Carles Dir. General,NaN,13/07/2020


In [17]:
#!pip install xlsxwriter

In [18]:
import pandas as pd
import xlsxwriter
import datetime
from google.colab import files

avui = datetime.datetime.now()
avui = str(avui).split(" ")[0].replace("-", "")

# Exemple de tres taules diferents
df1 = pd.DataFrame(RM)
df2 = pd.DataFrame(carrecs)
df3 = pd.DataFrame(carrecs_hist)

# Crear l'arxiu d'Excel i afegir-hi les tres taules en pestanyes diferents
with pd.ExcelWriter(avui+" "+societat+'-empresia.xlsx', engine='xlsxwriter') as writer:
    df1.to_excel(writer, sheet_name='BORME', index=False)
    df2.to_excel(writer, sheet_name='Càrrecs_'+avui, index=False)
    df3.to_excel(writer, sheet_name='Càrrecs_històric', index=False)

files.download(avui+" "+societat+'-empresia.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>